In [11]:
import cv2

# 이미지 로드
image = cv2.imread('/Users/mac/Dev/Project/ThermalCamera/Model/dataset/train/with_human/2023-09-11 05-33-46.mp4_4.png')

# 이미지의 차원 확인
height, width, channels = image.shape

# 채널 수에 따라 이미지 유형 확인
if channels == 1:
    print("이미지는 1채널 (그레이스케일) 이미지입니다.")
elif channels == 3:
    print("이미지는 3채널 (RGB) 이미지입니다.")
else:
    print("이미지의 채널 수가 1 또는 3이 아닙니다.")


이미지는 3채널 (RGB) 이미지입니다.


In [83]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 하이퍼파라미터 설정
image_height = 24
image_width = 32
batch_size = 32
epochs = 5


1. 데이터 전처리

In [84]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# 데이터셋 경로 설정
dataset_dir = '/Users/mac/Dev/Project/ThermalCamera/Model/dataset/'
with_human_dir = os.path.join(dataset_dir, 'train', 'with_human')
without_human_dir = os.path.join(dataset_dir, 'train', 'without_human')
# print(with_human_dir)
# 이미지 파일 목록 읽기
with_human_images = [os.path.join(with_human_dir, filename) for filename in os.listdir(with_human_dir)]
without_human_images = [os.path.join(without_human_dir, filename) for filename in os.listdir(without_human_dir)]
# print(with_human_images)
# 이미지 파일 경로와 레이블(1 또는 0)을 저장할 리스트 초기화
data = []
labels = []

# 'with_human' 이미지와 레이블(1) 추가
for image_path in with_human_images:
    if not image_path.endswith('.DS_Store'):  # .DS_Store 파일 건너뛰기
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # 이미지를 흑백으로 읽어옴
        image = cv2.resize(image, (image_width, image_height))  # 이미지 크기 조절
        data.append(image)
        labels.append(1)

# 'without_human' 이미지와 레이블(0) 추가
for image_path in without_human_images:
    if not image_path.endswith('.DS_Store'):  # .DS_Store 파일 건너뛰기
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (image_width, image_height))
        data.append(image)
        labels.append(0)



# 데이터를 NumPy 배열로 변환
data = np.array(data, dtype=np.float32)
labels = np.array(labels)

# 데이터를 0에서 1 사이의 값으로 정규화
data /= 255.0

# 첫 번째 호출로 훈련 데이터와 검증 데이터 생성
train_data, validation_data, train_labels, validation_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# 두 번째 호출로 테스트 데이터 생성 (훈련 데이터를 기반으로 생성하지 않음)
data_temp, test_data, labels_temp, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# 데이터의 shape 확인
print("Train data shape:", train_data.shape)
print("Validation data shape:", validation_data.shape)
print("Test data shape:", test_data.shape)




/Users/mac/Dev/Project/ThermalCamera/Model/dataset/train/with_human
['/Users/mac/Dev/Project/ThermalCamera/Model/dataset/train/with_human/2023-09-11 05-33-46.mp4_136.png', '/Users/mac/Dev/Project/ThermalCamera/Model/dataset/train/with_human/2023-09-11 05-33-46.mp4_122.png', '/Users/mac/Dev/Project/ThermalCamera/Model/dataset/train/with_human/2023-09-11 06-28-34.mp4_8.png', '/Users/mac/Dev/Project/ThermalCamera/Model/dataset/train/with_human/2023-09-11 06-29-52.mp4_51.png', '/Users/mac/Dev/Project/ThermalCamera/Model/dataset/train/with_human/2023-09-11 06-08-27.mp4_24.png', '/Users/mac/Dev/Project/ThermalCamera/Model/dataset/train/with_human/2023-09-11 06-08-27.mp4_30.png', '/Users/mac/Dev/Project/ThermalCamera/Model/dataset/train/with_human/2023-09-11 06-08-27.mp4_18.png', '/Users/mac/Dev/Project/ThermalCamera/Model/dataset/train/with_human/2023-09-11 06-28-34.mp4_36.png', '/Users/mac/Dev/Project/ThermalCamera/Model/dataset/train/with_human/2023-09-11 06-28-34.mp4_22.png', '/Users/mac/

2. CNN 모델 아키텍쳐 정의

In [85]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 1)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model training

In [86]:
model.fit(
    train_data,  # 훈련 데이터
    train_labels,  # 훈련 레이블
    epochs=epochs,  # 에포크 횟수
    validation_data=(validation_data, validation_labels),  # 검증 데이터
    batch_size=batch_size  # 배치 크기
)


Epoch 1/5


loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0783246a-4091-11ee-8fca-aead88ae2785/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0783246a-4091-11ee-8fca-aead88ae2785/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0783246a-4091-11ee-8fca-aead88ae2785/Library/Caches/com.apple.xbs/Sources/MetalPerformanceSh

83/85 [============================>.] - ETA: 0s - loss: 0.1051 - accuracy: 0.9665

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0783246a-4091-11ee-8fca-aead88ae2785/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x12x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0783246a-4091-11ee-8fca-aead88ae2785/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x12x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0783246a-4091-11ee-8fca-aead88ae2785/Library/Caches/com.apple.xbs/Sources/MetalPerformanceSh

85/85 [==============================] - ETA: 0s - loss: 0.1039 - accuracy: 0.9670

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0783246a-4091-11ee-8fca-aead88ae2785/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0783246a-4091-11ee-8fca-aead88ae2785/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0783246a-4091-11ee-8fca-aead88ae2785/Library/Caches/com.apple.xbs/Sources/MetalPerformanceSh

85/85 [==============================] - 14s 106ms/step - loss: 0.1039 - accuracy: 0.9670 - val_loss: 0.0079 - val_accuracy: 0.9985
Epoch 2/5
85/85 [==============================] - 1s 16ms/step - loss: 0.0090 - accuracy: 0.9974 - val_loss: 0.0026 - val_accuracy: 0.9985
Epoch 3/5
85/85 [==============================] - 1s 16ms/step - loss: 0.0043 - accuracy: 0.9981 - val_loss: 0.0062 - val_accuracy: 0.9985
Epoch 4/5
85/85 [==============================] - 1s 16ms/step - loss: 0.0187 - accuracy: 0.9944 - val_loss: 0.0086 - val_accuracy: 0.9956
Epoch 5/5
85/85 [==============================] - 1s 16ms/step - loss: 0.0078 - accuracy: 0.9978 - val_loss: 0.0027 - val_accuracy: 1.0000


모델 평가

In [87]:
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test accuracy: {test_accuracy}")


22/22 [==============================] - 0s 9ms/step - loss: 0.0027 - accuracy: 1.0000
Test accuracy: 1.0


In [91]:
import cv2
import numpy as np

# 이미지 파일 경로 설정
img_path = '/Users/mac/Dev/Project/ThermalCamera/Model/dataset/train/with_human/2023-09-11 05-33-46.mp4_28.png'

# 이미지 읽기 및 전처리
image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # 이미지를 흑백으로 읽어옴
image = cv2.resize(image, (image_width, image_height))  # 이미지 크기 조절
image = image.astype(np.float32) / 255.0  # 정규화

# 모델에 이미지 입력
predictions = model.predict(np.expand_dims(image, axis=0))  # 이미지를 배치 차원을 추가하여 모델에 전달

print(predictions)


1/1 [==============================] - 0s 18ms/step
[[1.]]


모델 저장

In [93]:
model.save("detect.h5")  # 모델 저장